## Cluster Analysis

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

import datetime as dt
import matplotlib.pyplot as plt

### Import Data

In [2]:
charging_sessions_data = pd.read_csv("cleaned_data.csv")
weather_data = pd.read_csv("weather_burbank_airport.csv")

charging_sessions_data.head(5)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,...,kWhRequested,milesRequested,minutesAvailable,requestedDeparture,hour,weekday,quarter,session_duration,idle_time,energy_ratio
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,...,25.0,100.0,463.0,"Thu, 02 Jan 2020 20:51:54 GMT",13,3,1,6.039167,1.661111,1.000640
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,...,70.0,250.0,595.0,"Thu, 02 Jan 2020 23:31:50 GMT",13,3,1,9.025278,2.337778,0.472814
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,...,8.0,20.0,60.0,"Thu, 02 Jan 2020 14:56:35 GMT",13,3,1,10.713056,8.071111,0.815125
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,...,8.0,20.0,65.0,"Thu, 02 Jan 2020 15:04:58 GMT",13,3,1,2.644722,1.331667,0.294375
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,...,16.0,40.0,504.0,"Thu, 02 Jan 2020 22:24:01 GMT",14,3,1,8.144167,3.852778,0.835938


### Cluster Analysis of Archetypical Charging Events
1. **Define possible Archetypical Charging Events**
* Identify archetypical charging events to do the clusters around
* Define appropriate names for these clusters
2. **Identify Best Approach**
* Test the unsupervised machine learning method options to do the analysis
    * K-means
    * K-medoid
3. **Validate the Clusters**
* Evaluate the accuracy and performance of the cluster analysis
    * Intracluster distance (distance between data points inside the cluster)
    * Intercluster distance (distance between data points in different clusters)

### 1. Define possible Archetypical Charging Events

### 2. Identify Best Approach

### 3. Validate the Clusters